<hr style="height:1px">
<hr style="height:3px">

#   Data Challenge : Analyzing Food-Mart Data


<hr style="height:1px">
<hr style="height:3px">


## Instructions and the Problem Statement : 

In raw_data folder, you should find 4 inter-related datasets: a dataset of **transactions**, a dataset of **products**, a dataset of **product_classes**, and a dataset with information about **promotions**. The original source is a freely available Foodmart dataset, downloaded from [here](https://sites.google.com/a/dlpage.phi-integration.com/pentaho/mondrian/mysql-foodmart-database/foodmart_mysql.tar.gz). You might need to do some cleaning to the data to get it into a workable state.

**Question 1**

Management is trying to better understand what is happening at these food marts. To that end, please create some visualizations to help us better understand the following dimensions of the Foodmart data:
- Category of products sold
- Brand of products sold

**Question 2**

Foodmart is developing their strategy for the next year. One of their goals is to increase sales and profitability. Please provide the executive team with insights from the data that highlight growth opportunities. Here are some ideas to get you started:

- Expand product offerings: which products or categories should we expand and why?
- Offer more promotions: how should we target our promotions and why?
- Market to a certain customer segment: what segment of customers should we market to and how?

Feel free to think of other ideas. Please focus on only 1 to 2 opportunities and provide an executive summary of your recommendations, the potential impact, and what data you found to support your strategy.


## Tools used and the Solution Sequence : 

ALL analysis for the given problem was done using a combination of **Python** and **SQL** (PostGreSQL). The list of all packages installed in the **virtual environment** where the codes were built has been included in the folder as **requirements.txt** and can be extracted if needed. 

While those SQL queries that produce an output table can be run from within the ipython notebook - some queries that produce new tables or produce row counts etc for verification purposes need to be run outside from the console after logging into the database.

The solution has been split into two iPython Notebooks:

- [Data_Visualization.ipynb](Data_Visualization.ipynb)  --(Question#1)
        
- [Sales_Analysis.ipynb](Sales_Analysis.ipynb)      --(Question#2)

An **executive summary** for the entire analysis has been included near the end of the Sales_Analysis notebook.

In this notebook I have outlined the initial steps taken to :  

- Clean the Data
- Create a SQL database and 
- Load the data onto SQL tables 


<hr style="height:1px">

In [26]:
# Import necessary packages
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
pd.set_option('notebook_repr_html',False)

## STEP 1 : 
### Cleaning the given data tables : 

The following steps were taken to clean the data : 

- File headers had whitespace between column names which had to be removed
- Many columns with string data types had quotes('') encasing the variables - these were removed
- The promotions file had a column with multiple comma-separated entries which needed special treatment to be read into a pandas data frame
- Some numeric columns in transaction table had string data types that needed conversion
- Promotion table had one NULL entry which we removed from our analysis 
- Transaction table had NULL values in the last column which was a variable we did not need for our analysis
- Start and End date columns in promotion table were converted from string to datetime datatype

The dataframes were then loaded into the Database : 

$$ CSV -> Data Frame (cleaning) -> SQL-Table $$

In [38]:
# Function to remove quotes from variables and convert numbers from strings

def cleaner(acolumn):
    try:
        cleaned = acolumn.str.replace('\'','')
    except:
        cleaned = acolumn
    return cleaned

def data_cleaner(filename):
    tname = pd.read_csv(filename)
    colname = []
    for item in tname.columns:
        colname.append(item)
    for col in colname:
        tname[col] = cleaner(tname[col])
    return tname

In [39]:
# Making the tables from csv files : 

product = data_cleaner('product.csv')
product_class = data_cleaner('product_class.csv')
transactions = data_cleaner('transactions.csv')

# Making the promotion table separately : 

promotion = pd.read_csv('promotion.csv', quotechar="'")
for item in promotion.columns:
    promotion[item] = cleaner(promotion[item])
    

In [40]:
# Converting string datatype in some columns to numbers
transactions[['store_sales','store_cost','unit_sales']] = transactions[['store_sales','store_cost','unit_sales']].apply(pd.to_numeric)

In [41]:
# Function to check for null values in the tables

def check_4_null(tablename):
    null_position = pd.isnull(tablename).any(1).nonzero()
    if tablename.iloc[null_position].size == 0 :
        print 'No Nulls'
    else:
        print 'Need to find nulls'         

In [42]:
list_of_tables = [product,product_class,transactions,promotion]
for tables in list_of_tables:
    check_4_null(tables)

No Nulls
No Nulls
Need to find nulls
Need to find nulls


In [43]:
promotion = promotion.dropna()

### A new variable - Profit

I defined a new variable Profit and added it to the transactions table. 
The following form of profit equation was used for all analysis henceforth :  

            $$   Profit =  { (Store     Sales - Store     Cost) } * { Unit Sales } $$
          


In [45]:
transactions['profit'] = (transactions.store_sales - transactions.unit_sales)*transactions.store_cost  

In [9]:
# For the initial date-analysis the start_date and end_date columns in tables were not in the correct format 
# The convertion was done in SQL - this was run inside database

sql_query = """UPDATE promotion SET end_date=to_date(end_date, 'YYYY-MM-DD');"""  

## STEP 2 : 
### Making SQL Database and loading tables



In [2]:
# Create a Database for all analysis
! createdb etsy

In [10]:
# Setting up connection using SQL Alchemy : 
dbname = 'etsy'
username = 'parama'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))

In [11]:
# Setting up connection using Psycopg2 : 
try:
    conn = psycopg2.connect(database = 'etsy', user = 'parama' , password = '###')
except:
    print 'Unable to connect'

In [22]:
def sql_table_maker(tname,name):    
    tname.to_sql(name,engine)
    print name + ' was created!'    

In [24]:
list_of_tables = [product,product_class,transactions,promotion]
list_of_tablenames = ['product','product_class','transactions','promotion']
for index in range(4):
    sql_table_maker(list_of_tables[index],list_of_tablenames[index])

product was created!
product_class was created!
transactions was created!
promotion was created!
